In [ ]:
import pandas as pd
%reload_ext autoreload
%autoreload 2

import sys
 
# setting path
sys.path.append('../shared')

from progress import ProgressEstimator
from progsnap import ProgSnap2Dataset
from progsnap import PS2
from progsnap import EventType

In [ ]:
submit_columns = [EventType.Submit, EventType.RunProgram, 'Project.Submit']
problem_id = 3
problem_id_column = PS2.ProblemID
code_column = PS2.Code
data_folder = "data/pcrs-f18/"

In [ ]:
from preprocess import SimpleAIFBuilder

data = ProgSnap2Dataset(data_folder)
submissions = SimpleAIFBuilder.get_submissions_table(data, submit_columns)
submissions.head()

In [ ]:
def extract_starter_code(code):
    lines = code.split("\n")
    starter = ""
    comment_lines = 0
    for line in lines:
        if line.strip().startswith('"""'):
            comment_lines += 1
        starter += line + "\n"
        if comment_lines == 2:
            break
    if comment_lines < 2:
        return ""
    return starter

# extract_starter_code(code[0])

In [ ]:
def extract_problem_starter_code(all_code):
    samples = all_code[:50]
    common_code = pd.Series([extract_starter_code(code) for code in samples]).value_counts().sort_values(ascending=False)
    return common_code.index[0]

# extract_problem_starter_code(code)

In [ ]:
problem_ids = submissions[problem_id_column].unique()
starter_code = []
for problem_id in problem_ids:
    assignment_submissions = submissions[submissions[problem_id_column] == problem_id]
    assignment_code = SimpleAIFBuilder.get_code_table(data, assignment_submissions, problem_id_column, code_column)["Code"]
    starter_code.append(extract_problem_starter_code(assignment_code))

starter_code = pd.DataFrame({problem_id_column: problem_ids, "StarterCode": starter_code})
starter_code

In [ ]:
starter_code.to_csv(data_folder + "LinkTables/Problem.csv", index=False)